<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mortage-Default-Prediction" data-toc-modified-id="Mortage-Default-Prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mortage Default Prediction</a></span></li></ul></div>

## Mortage Default Prediction

In [1]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-ddd47da4-acfa-4590-babf-ab051b7f77b6',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'kCxdy2mVdTMcTotMupUlOh_cc6Wrynmh4b7kIAIZ98AU'
}

configuration_name = 'os_f99601c62bd54655b8bdd838cc2ad12e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(cos.url('Mortgage_Training Data.csv', 'pyrtraining-donotdelete-pr-8zy0j2en6j4jea'))
data.take(5)


ModuleNotFoundError: No module named 'ibmos2spark'

In [3]:
import pandas as pd

In [2]:
pandasDF=data.toPandas()
pandasDF.head()

NameError: name 'data' is not defined

In [13]:
data.dtypes

[('ID', 'double'),
 ('Income', 'double'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'double'),
 ('YearsCurrentEmployer', 'double'),
 ('NumberOfCards', 'double'),
 ('CCDebt', 'double'),
 ('Loans', 'double'),
 ('LoanAmount', 'double'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

In [18]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier


SI1 = StringIndexer(inputCol='AppliedOnline', outputCol='AppliedOnlineEncoded')
SI2 = StringIndexer(inputCol='Residence',outputCol='ResidenceEncoded')
SI3 = StringIndexer(inputCol='Location',outputCol='LocationEncoded')
labelIndexer = StringIndexer(inputCol='MortgageDefault', outputCol='label').fit(data)

OH1 = OneHotEncoder(inputCol="AppliedOnlineEncoded", outputCol="AppliedOnlineEncoded"+"classVec")
OH2 = OneHotEncoder(inputCol="ResidenceEncoded", outputCol="ResidenceEncoded"+"classVec")
OH3 = OneHotEncoder(inputCol="LocationEncoded", outputCol="LocationEncoded"+"classVec")

assembler = VectorAssembler(inputCols=["Income", "AppliedOnlineEncoded"+"classVec",\
                                       "ResidenceEncoded"+"classVec", "YearCurrentAddress",\
                                       "YearsCurrentEmployer", "NumberOfCards", "CCDebt", \
                                       "Loans", "LoanAmount", "SalePrice", "LocationEncoded"+"classVec"],\
                            outputCol="features")

In [19]:
rf=RandomForestClassifier(labelCol="label", featuresCol="features")

labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,labelIndexer,OH1,OH2,OH3,assembler,rf,labelConverter])

In [20]:
train, test = data.randomSplit([0.8,0.2], seed=6)
train.cache()
test.cache()

DataFrame[ID: double, Income: double, AppliedOnline: string, Residence: string, YearCurrentAddress: double, YearsCurrentEmployer: double, NumberOfCards: double, CCDebt: double, Loans: double, LoanAmount: double, SalePrice: int, Location: int, MortgageDefault: string]

In [21]:
model = pipeline.fit(train)

In [27]:
results=model.transform(test)
results.printSchema()

root
 |-- ID: double (nullable = true)
 |-- Income: double (nullable = true)
 |-- AppliedOnline: string (nullable = true)
 |-- Residence: string (nullable = true)
 |-- YearCurrentAddress: double (nullable = true)
 |-- YearsCurrentEmployer: double (nullable = true)
 |-- NumberOfCards: double (nullable = true)
 |-- CCDebt: double (nullable = true)
 |-- Loans: double (nullable = true)
 |-- LoanAmount: double (nullable = true)
 |-- SalePrice: integer (nullable = true)
 |-- Location: integer (nullable = true)
 |-- MortgageDefault: string (nullable = true)
 |-- AppliedOnlineEncoded: double (nullable = false)
 |-- ResidenceEncoded: double (nullable = false)
 |-- LocationEncoded: double (nullable = false)
 |-- label: double (nullable = false)
 |-- AppliedOnlineEncodedclassVec: vector (nullable = true)
 |-- ResidenceEncodedclassVec: vector (nullable = true)
 |-- LocationEncodedclassVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)

In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline

In [95]:
df = pd.read_csv('Mortgage_Training Data.csv')

In [96]:
df.head()

,ID,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location,MortgageDefault
0,100522.0,43982.0,YES,Owner Occupier,13.0,11.0,2.0,1055.0,0.0,9405.0,500000,110,NO
1,101756.0,59944.0,YES,Owner Occupier,20.0,11.0,2.0,3894.0,0.0,9880.0,750000,110,NO
2,101354.0,57718.0,YES,Owner Occupier,25.0,16.0,2.0,1555.0,1.0,6285.0,155000,130,YES
3,100512.0,45621.0,YES,Owner Occupier,1.0,19.0,1.0,1878.0,0.0,9260.0,195000,100,YES
4,100537.0,45081.0,NO,Owner Occupier,14.0,15.0,2.0,713.0,1.0,8430.0,140000,110,NO


In [97]:
df['AppliedOnline'].value_counts()

YES    285
NO     134
Name: AppliedOnline, dtype: int64

In [98]:
df['AppliedOnline'] = np.where(df['AppliedOnline'] == 'YES', 1, 0 )

In [99]:
df['MortgageDefault'] = np.where(df['MortgageDefault'] == 'YES', 1, 0)

In [100]:
df = pd.get_dummies(df)

In [101]:
X = df.drop(['ID', 'MortgageDefault'], axis=1)
y = df['MortgageDefault']

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [114]:
ss = StandardScaler(with_mean=False)

pipe = make_pipeline(StandardScaler(), RandomForestClassifier())

In [104]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('randomforestclassifier', RandomForestClassifier())])

In [105]:
pipe.score(X_test, y_test)

0.7047619047619048

In [120]:
params = {
    'n_estimators': [50, 100, 250, 500],
    'criterion': ['gini', 'entropy'],
}

In [121]:
gs = GridSearchCV(pipe, params)

In [122]:
gs.fit(X_train, y_train)

ValueError: Invalid parameter criterion for estimator Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())]). Check the list of available parameters with `estimator.get_params().keys()`.